### Obtencion de datos 
--- 
A partir de los boletines disponibles en: https://presidente.gob.mx/sala-de-prensa/boletines/

In [ ]:
# Remover comentario si es requerido:
#!pip install selenium bs4 html2text
import pandas as pd
from selenium import webdriver
import json
from bs4 import BeautifulSoup
import re
import time

#### Se inicia el navegador
--- 

In [ ]:
# Utilice el que sea requerido según su caso.
'''
driver = webdriver.ChromiumEdge(executable_path='msedgedriver.exe')
driver = webdriver.Firefox(executable_path='/path/to/geckodriver')
driver = webdriver.Chrome(executable_path='chromedriver.exe')
'''
driver = webdriver.Firefox()
driver.get('https://presidente.gob.mx/sala-de-prensa/boletines/')

# Función para obtener los links de los boletines de prensa dependiendo de la página en la que se encuentre
def navegacion(num):
    if( num >= 2):
        driver.get('https://presidente.gob.mx/sala-de-prensa/boletines/page/'+ str(num) +'/')
    if (num == 1):
        driver.get('https://presidente.gob.mx/sala-de-prensa/boletines/')

    return driver

#### Se itera en los articulos de cada pagina y se guarda en un archivo_con_articulos de texto
--- 

In [1]:
def escribir_articulos(archivo,titulo,fecha,articulo,link):
    # Escribir el título y contenido del artículo en el archivo_con_articulos de texto
    archivo.write(",\n{\n")
    archivo.write(f"\"link\": \"{link}\",\n")
    archivo.write(f"\"Fecha\": \"{fecha}\",\n")
    archivo.write(f"\"Título\": \"{titulo}\",\n")
    archivo.write(f"\"Contenido\": \"{articulo}\"")
    archivo.write("\n}")


# Abrir un archivo_con_articulos de texto para escribir los datos recolectados en cada una de las paginas de los articulos
rango = 800
with open('./datos_crudos_.json', 'w', encoding='utf-8') as archivo:
    archivo.write("[")
    for num in range(1,rango):
        try:
            navegacion(num)
            print("pagina: " + driver.current_url)
            
            # Obtener el HTML de la página con BeautifulSoup
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            
            # Obtener todos los artículos de la página en la que se encuentra y su fecha
            articulos = soup.find_all('h3', {'class': 'entry-title'})
            fecha_articulo = soup.find('span', {'class': 'posted-on'}).text.strip()
            
            for articulo in articulos:
                enlace = articulo.find('a')['href']
                driver.get(enlace)
                soup_articulo = BeautifulSoup(driver.page_source, 'html.parser')
                titulo_articulo = soup_articulo.find('h2', {'class': 'entry-title'}).text.strip()
                contenido_articulo = soup_articulo.find('div', {'class': 'entry-content'})
                contenido_articulo =contenido_articulo.get_text()
                contenido_articulo =re.sub('"',' ',contenido_articulo)
                contenido_articulo =re.sub('\n',' ',contenido_articulo)
                contenido_articulo =re.sub('\t',' ',contenido_articulo)
                contenido_articulo =contenido_articulo.strip()
                
                escribir_articulos(archivo,titulo_articulo,fecha_articulo,contenido_articulo,enlace)

            print("ciclo de boletines: " + str(num))

        except Exception as e: 
            print(e)
            continue
        
    archivo.write("\n]")        

# Cerrar el driver al finalizar y cerrar el archivo 
# Mientras se esten haciendo pruebas, se recomienda comentar estas dos lineas
if (rango >= 30):
    driver.quit()
    archivo.close()

pagina: https://presidente.gob.mx/sala-de-prensa/boletines/
ciclo de boletines: 1


In [2]:
with open('./datos_crudos_.json', 'r') as file_1:
    json_res: dict = json.loads(file_1.read(),strict=False)
corpus = pd.DataFrame(json_res)
print(corpus.head())

                                                link  ...                                          Contenido
0  https://presidente.gob.mx/presidente-destaca-f...  ...  2023: Año de Francisco Villa, el revolucionari...
1  https://presidente.gob.mx/federacion-analiza-c...  ...  2023: Año de Francisco Villa, el revolucionari...
2  https://presidente.gob.mx/presidente-llama-a-f...  ...  2023: Año de Francisco Villa, el revolucionari...
3  https://presidente.gob.mx/gobierno-federal-dec...  ...  2023: Año de Francisco Villa, el revolucionari...
4  https://presidente.gob.mx/presidente-llama-a-r...  ...  2023: Año de Francisco Villa, el revolucionari...

[5 rows x 4 columns]
